In [1]:
from pysat.solvers import Minisat22
from time import process_time




# function to transform cell-coordinations into numbers

def codify_cells(row, col, SIZE): 
    return row * (SIZE + 2) + col + 1     # we add "1" in the end because Minisat22 does not accept "0"


# function to transform neighbouring cells' coordinations into numbers

def codifying_neighbours(neighbours, SIZE): 
    neighbour_codes = []
    for (x, y) in neighbours:
        neighbour_codes.append(codify_cells(x, y, SIZE))
    return neighbour_codes


###---- Cardinality Constraints ----###


# first we consider 8 neighbours for every cell because we added two rows and columns to our grid which have no mines 

num_of_neighbours = 8 

# cell constraint function for exactly "cell_value" mines in neighbouring cells

def exactly(cell_value, neighbours): 
    if cell_value == 0:
        return [[-neighbours[i]] for i in range(num_of_neighbours)]  # adds not-mine clauses when cell value is "0"

    most = at_most(cell_value, neighbours)
    least = at_least(cell_value, neighbours)
    
    return most + least                          

# at most "cell_value" mines in neighbouring cells

def at_most(cell_value, neighbours):
    clauses = []
    
    interpretations = 2 ** num_of_neighbours
    for I in range(interpretations):
        binary_format= '{0:08b}'.format(I, num_of_neighbours)   # shows binary format of all possible interpretations
        mine_count = binary_format.count("1")                 
        if mine_count == cell_value + 1:
            clauses.append([-neighbours[i] for i in range(num_of_neighbours) if (binary_format[i] == '1')])
           
    return clauses

# at least "cell_value" mines in neighbouring cells

def at_least(cell_value, neighbours):
    clauses = []
    
    interpretations = 2 ** num_of_neighbours
    for I in range(interpretations):
        binary_format = '{0:08b}'.format(I, num_of_neighbours)   # shows binary format of all possible interpretations
        mine_count = binary_format.count("1") 
        if mine_count == (num_of_neighbours - (cell_value - 1)):
            clauses.append([neighbours[i] for i in range(num_of_neighbours) if (binary_format[i] == '1')])

    return clauses


###---- Minisat Solve ----###

t_start = process_time()

def cell_solve(grid, row, column, safe_cells=[]):
    S = Minisat22()
    SIZE = len(grid)
    neighbour_clauses = []         # we will store the neighbouring clauses for each cell here
                                   # to be added to our SAT solver
    
    # clauses for two extra rows and columns added to our grid which have no mines 
    for c in range(SIZE + 2):
        S.add_clause([-codify_cells(0, c, SIZE)])
        S.add_clause([-codify_cells(SIZE + 1, c, SIZE)])
    for r in range(SIZE + 2):
        S.add_clause([-codify_cells(r, 0, SIZE)])
        S.add_clause([-codify_cells(r, SIZE + 1, SIZE)])

    for x in range(1, SIZE + 1):
        for y in range(1, SIZE + 1):
            cell = grid[x - 1][y - 1]
            if cell != '#':
                S.add_clause([-codify_cells(x, y, SIZE)])
                neighbours = [[x - 1, y - 1], [x - 1, y], [x - 1, y + 1],
                              [x    , y - 1],             [x    , y + 1],
                              [x + 1, y - 1], [x + 1, y], [x + 1, y + 1]]
                neighbours_codes = codifying_neighbours(neighbours, SIZE)
                clauses = exactly(int(cell), neighbours_codes)
                
                neighbour_clauses.extend(clauses)

    S.append_formula(neighbour_clauses)   # adding the neighbour clauses into the solver
    
    S.add_clause([codify_cells(row, column, SIZE)])
   
    return S.solve()

# defining a 9x9 grid (middle stage of the game)
grid = [ ['0', '0', '0', '0', '1', '#', '#', '#', '#'],
         ['0', '0', '0', '0', '1', '2', '#', '#', '#'],
         ['0', '1', '1', '1', '0', '1', '1', '#', '#'],
         ['0', '1', '#', '1', '0', '0', '1', '#', '#'],
         ['1', '2', '2', '1', '0', '0', '1', '#', '#'],
         ['1', '#', '1', '1', '1', '1', '1', '#', '#'],
         ['1', '1', '1', '1', '#', '#', '#', '#', '#'],
         ['0', '0', '1', '3', '#', '#', '#', '#', '#'],
         ['0', '0', '1', '#', '#', '#', '#', '#', '#']]

safe_cells = []
for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == '#':
            if not cell_solve(grid, i + 1, j + 1):
                safe_cells.append((i + 1, j + 1))

if cell_solve(grid, -1, -1, safe_cells):    # when we reach to the end of grid and want to start from top left corner again
    print("safe cells positions: ")
    print(safe_cells)
else:
    print("We are stuck! Can not define a 100% safe cell.")
    
t_stop = process_time()
duration_time = t_stop - t_start
print("CPU time:", duration_time)

safe cells positions: 
[(1, 7), (2, 8), (3, 8), (4, 8), (6, 8), (7, 6), (7, 7), (7, 8), (8, 5)]
CPU time: 0.421875
